In [1]:
import os
import random
import shutil
import sys
import yaml
from pathlib import Path

In [2]:
# Change to a different directory
os.chdir(r'C:\Users\SIU856522160\Downloads')
# Get the new default path
os.getcwd()

'C:\\Users\\SIU856522160\\Downloads'

In [3]:
num_batches = 20
folder_path = r".\\100k"
source_folder = folder_path
# Destination folder path w     here the selected images will be copied
destination_folder = r".\\data_batches_test"

In [4]:
def del_folders_and_files(destination_folder):

    if os.path.exists(destination_folder):
        for root, dirs, files in os.walk(destination_folder):
            for dir in dirs:
                subfolder_path = os.path.join(root, dir)
                try:
                    shutil.rmtree(subfolder_path)
                    print(f"Deleted subfolder: {subfolder_path}")
                except OSError as e:
                    print(f"Failed to delete subfolder: {subfolder_path} - {e}")
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted file: {file_path}")
                except OSError as e:
                    print(f"Failed to delete file: {file_path} - {e}")
        print("Deleted sub folders and files in the destination folder")
    else:
        print("Destination folder does not exist")

In [5]:
def create_batches(source_folder, num_batches, destination_folder, folder_name, train_folder_path):
    if "train" in folder_name.lower() or "val" in folder_name.lower() or "test" in folder_name.lower():
        print(f"Folder is {folder_name}")
        folder_name = folder_name.lower()
    else:
        print("Folder is not train or val")
        return "given the correct folder name"
    
    # Check if the destination folder exists, create it if not
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        
    # Skip deletion if folder_name is "val" or "test"
    if folder_name.lower() not in ["val", "test"]:
        del_folders_and_files(destination_folder)

    # List all sub-batch files in the train folder
    sub_batch_files = [file for file in os.listdir(train_folder_path) if file.startswith("Sub")]
    print(sub_batch_files)
    sorted_file_names = sorted(sub_batch_files, key=lambda x: int(x.split("_")[3]))
    print(sorted_file_names)
    for batch_num, sub_batch_file in enumerate(sorted_file_names, start=1):
        print(sub_batch_file)
        batch_folder = os.path.join(destination_folder, f"SubData_{batch_num}")
        if not os.path.exists(batch_folder):
            os.makedirs(batch_folder)
        image_destination = os.path.join(batch_folder, "images", folder_name)
        label_destination = os.path.join(batch_folder, "labels", folder_name)
        if not os.path.exists(image_destination):
            os.makedirs(image_destination)
        if not os.path.exists(label_destination):
            os.makedirs(label_destination)

        # Read the sub-batch file for the current batch
        sub_batch_file_path = os.path.join(train_folder_path, sub_batch_file)
        with open(sub_batch_file_path, "r") as sub_batch_file:
            selected_files = sub_batch_file.read().splitlines()

        # Process the selected files for the current batch
        for file_name in selected_files:
            image_source_path = os.path.join(source_folder, folder_name, file_name)
            image_destination_path = os.path.join(batch_folder, "images", folder_name, file_name)
            print(image_source_path)
            try:
                shutil.copy2(src=image_source_path, dst=image_destination_path)
                print(f"Moved image: {file_name}")
            except FileNotFoundError:
                print(f"Image not found: {file_name}")
                continue

            label_name = file_name.split(".")[0] + ".txt"
            label_source_path = os.path.join(source_folder, folder_name, label_name)
            label_destination_path = os.path.join(batch_folder, "labels", folder_name, label_name)
            if os.path.exists(label_source_path):
                try:
                    shutil.copy(src=label_source_path, dst=label_destination_path)
                    print(f"Moved label: {label_name}")
                except FileNotFoundError:
                    print(f"Label not found: {label_name}")
                    continue
                except OSError as e:
                    print(f"Error occurred while copying label: {label_name} - {e}")
                    continue


        # Create a subfolder named 'sub_date_names' and save the copied_images_file_img in it
        sub_date_names_folder = os.path.join(batch_folder, 'sub_date_names', folder_name)
        print(sub_date_names_folder)
        
        # Skip deletion if folder_name is "val" or "test"
        if folder_name.lower() not in ["val", "test"]:
            if not os.path.exists(sub_date_names_folder):
                os.makedirs(sub_date_names_folder)
            
            if not any(os.listdir(sub_date_names_folder)): 
                del_folders_and_files(sub_date_names_folder)

        # Create a YAML file in the batch folder with the provided configuration
        yaml_data = {
            'names': [
                "person", "rider", "car", "truck", "bus", "train", "motor", "bike", "traffic light", "traffic sign"
            ],
            'path': './data/vech',
            'train': f'./SubData_saved_img_{batch_num}_train.txt',
            'val': f'./SubData_saved_img_{batch_num}_val.txt',
            'nc': 10,
        }
        
        # Reverse the order of keys before writing to the YAML file
        yaml_data = {k: yaml_data[k] for k in reversed(yaml_data)}
        
        yaml_file_path = os.path.join(batch_folder, f"batch_info_{batch_num}.yaml")
        with open(yaml_file_path, 'w') as yaml_file:
            yaml.dump(yaml_data, yaml_file, default_flow_style=False)


In [6]:
# create_batches(source_folder, num_batches, destination_folder,folder_name="Train")
with open(r'.\\major\logs\batches_create_train_temp.txt', 'w') as file:
    # Redirect the standard output to the file
    sys.stdout = file
    # create_batches(source_folder, num_batches, destination_folder, folder_name="test", train_folder_path=r"C:\Users\SIU856522160\Downloads\major\dataset\dataset_file_names\sub_date_names\test")
    create_batches(source_folder, num_batches, destination_folder, folder_name="train", train_folder_path=r"C:\Users\SIU856522160\Downloads\major\dataset\dataset_file_names\sub_date_names\train")
    # Reset the standard output to the console
    sys.stdout = sys.__stdout__

In [ ]:
# create_batches(source_folder, num_batches, destination_folder,folder_name="Train")
with open(r'.\\major\logs\batches_create_val_temp.txt', 'w') as file:
    # Redirect the standard output to the file
    sys.stdout = file    
    create_batches(source_folder, num_batches, destination_folder, folder_name="val", train_folder_path=r"C:\Users\SIU856522160\Downloads\major\dataset\dataset_file_names\sub_date_names\val")
    # Reset the standard output to the console
    sys.stdout = sys.__stdout__

In [ ]:
# create_batches(source_folder, num_batches, destination_folder,folder_name="Train")
with open(r'.\\major\logs\batches_create_test_temp.txt', 'w') as file:
    # Redirect the standard output to the file
    sys.stdout = file
    create_batches(source_folder, num_batches, destination_folder, folder_name="test", train_folder_path=r"C:\Users\SIU856522160\Downloads\major\dataset\dataset_file_names\sub_date_names\test")
    # Reset the standard output to the console
    sys.stdout = sys.__stdout__